In [1]:
# data visualiztion
import matplotlib.pyplot as plt
import seaborn as sns

# raw data handling
import pandas as pd
import numpy as np
import datetime as dt

# for linear regression models
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms 
from statsmodels.formula.api import ols
import scipy.stats as stats

# recursive feature elimination (w/ cross validation), linear regression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

# Variance inflation factor, mean abs/squarred error
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings # weird sns.distplot() warnings
warnings.filterwarnings("ignore")

# make stuff look cooler
plt.style.use('fivethirtyeight')

In [48]:
# creates dictionary of variance inflation factors. 
def create_vif_list(X):
    X = sm.add_constant(X)
    vif_dict = {}

    for i in range(len(X.columns)):
        vif = variance_inflation_factor(X.values, i)
        v = X.columns[i]
        vif_dict[v] = vif

    good_vifs = []
    bad_vifs = []

    for k,v in vif_dict.items():
        if v < 10:
            good_vifs.append(k)
        else:
            bad_vifs.append(k)

    return good_vifs,bad_vifs

def create_vif_dictionary(X):
    X = sm.add_constant(X)
    vif_dict = {}

    for i in range(len(X.columns)):
        vif = variance_inflation_factor(X.values, i)
        v = X.columns[i]
        vif_dict[v] = vif

    return vif_dict

# create a dictionary showing the adjusted R-squared values for each feature individually
def create_R2_dictionary(X,y):

    adj_R_squares = {}

    for feature in X.columns:
        predictors_int = sm.add_constant(X[feature])
        model = sm.OLS(y,predictors_int).fit()
        adj_R_square = float(model.summary2().tables[0][3][0])
        adj_R_squares[feature] = adj_R_square
        
    return adj_R_squares

# author's docstring is in a markup cell down below in the stepwise selection section
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)

    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            worst_feature_name = included[worst_feature]
            included.pop(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature_name, worst_pval))
        if not changed:
            break
    return included

# recursive feature elimination
def run_RFE(X,y):

    linreg = LinearRegression()
    selector = RFE(linreg)
    selector = selector.fit(X, y)

    selected = selector.support_ # list of bools representing if feature is selected or not
    selections = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(selected): # append labels of selected features to selections list
        if feature == True:
            selections.append(labels[idx])
        else:
            pass
    
    ranked = selector.ranking_ # list of bools representing if feature is selected or not
    rankers = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(ranked): # append labels of selected features to selections list
        if feature == 1:
            rankers.append(labels[idx])
        else:
            pass
    return selections, rankers

# recuersive feature elimination with cross validation
def run_RFECV(X,y,select_rank=False):

    linreg = LinearRegression()
    selector = RFECV(linreg)
    selector = selector.fit(X, y)

    selected = selector.support_ # list of bools representing if feature is selected or not
    selections = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(selected): # append labels of selected features to selections list
        if feature == True:
            selections.append(labels[idx])
        else:
            pass
    
    ranked = selector.ranking_ # list of bools representing if feature is selected or not
    rankers = [] # list of column names that are selected
    labels = list(X.columns) # list of all column names 

    for idx,feature in enumerate(ranked): # append labels of selected features to selections list
        if feature == 1:
            rankers.append(labels[idx])
        else:
            pass

    if select_rank == True:
        return selections
    else:
        return rankers

# returns OLS linear regression model
def run_OLS_model(X,y):

    predictors_int = sm.add_constant(X)
    model = sm.OLS(y,predictors_int).fit()

    return model

# returns dictionary: keys are column labels, values are count of NaN values present
def create_NaN_dictionary(X):

    NaN_dict = {}

    for feature in X.columns:
        NaN_dict[feature] = sum(X[feature].isna())

    return NaN_dict



#### Set up dataframe and deal with some missing data

In [3]:
X = pd.read_csv('data/kc_house_data.csv')
X.drop(X.loc[X['sqft_basement']=='?'].index,inplace=True) # remove '?' from the data.
X.drop(['id','lat','long'],inplace=True,axis=1)

#### Eliminate target variable outliers
(for price column, drop anything more than 3 standard deviations from the mean)

In [4]:
std_thresh = X.price.std()*3 # three standard deviations
X = X.loc[abs(X['price']) <= std_thresh]

#### interpret tax bracket by zipcode according to average income from IRS 

In [5]:
income = pd.read_csv('data/irs_income_by_zipcode.csv')

income.avg_taxable_income = income.avg_taxable_income*1000 # scaling data back to dollar units (was in thousands)
income.taxable_income_amount = income.taxable_income_amount*1000 
income.rename({'taxable_income_amount':'zip_tax_revenue'},axis=1,inplace=True) # renaming to something more idiomatic

# # create lists to stage data in order to concat new columns later
single_filing_bracket = [] # for the single filing tax schedule
joint_filing_bracket = [] #for the joint filing tax schedule

# assign tax bracket based on average income
for average in income.avg_taxable_income:
    if average > 523600:
        single_filing_bracket.append(7)
    elif average <= 9950:
        single_filing_bracket.append(1)
    elif (average >= 9951) and (average <= 40525):
        single_filing_bracket.append(2)
    elif (average >= 40526) and (average <= 86375):
        single_filing_bracket.append(3)
    elif (average >= 86376) and (average <= 164924):
        single_filing_bracket.append(4)
    elif (average >= 164925) and (average <= 209425):
        single_filing_bracket.append(5)
    elif (average >= 209426) and (average <= 523600):
        single_filing_bracket.append(6)

for average in income.avg_taxable_income:
    if average > 628301:
        joint_filing_bracket.append(7)
    elif average <= 19900:
        joint_filing_bracket.append(1)
    elif (average >= 19901) and (average <= 81050):
        joint_filing_bracket.append(2)
    elif (average >= 81051) and (average <= 172750):
        joint_filing_bracket.append(3)
    elif (average >= 172751) and (average <= 329850):
        joint_filing_bracket.append(4)
    elif (average >= 329851) and (average <= 418850):
        joint_filing_bracket.append(5)
    elif (average >= 418851) and (average <= 628300):
        joint_filing_bracket.append(6)

# assign discovered data to new column
income['single_filing_tax_bracket'] = single_filing_bracket
income['joint_filing_tax_bracket'] = joint_filing_bracket

# create dictionary of zipcodes with corresponding tax bracket
# dictionary will be used to assign create similar columns in the principal data.
single_filing_tax_dict = {}
joint_filing_tax_dict = {}

# create ditionaries to later assign values to principal data
for row in income.iterrows():
    single_filing_tax_dict[row[1][0]] = row[1][3]
    joint_filing_tax_dict[row[1][0]] = row[1][4]

# concat columns to X containing tax bracket based on single and joint filing federal income tax schedule
X['single_filing_bracket'] = X.zipcode.replace(to_replace=single_filing_tax_dict) # single filing 
X['joint_filing_bracket'] = X.zipcode.replace(to_replace=joint_filing_tax_dict) # joint filing

#### Convert object type data into numeric type.

In [6]:
# convert all string types into np floats
X.sqft_basement = [float(sq) for sq in list(X.sqft_basement)]

# Replaces grade strings with numerics based on data dict. 
grade_raws = list(X.grade.unique())
# replaces a cell value with the int of the first character of its existing string
for raw in grade_raws:
    X.grade.replace(to_replace=raw,value=int(raw[0]),inplace=True)

# replaces condition objects with numerics based on data dict.
condition_dict = {'Poor':1,'Fair':2,'Average':3,'Good':4,'Very Good':5}
for key in condition_dict:
    X.condition.replace(to_replace=condition_dict,inplace=True)

# replace yr_built NaNs with numeric 0
X.yr_renovated.replace(to_replace=np.nan,value=0,inplace=True)

In [7]:
# convert waterfront into numeric boolean
waterfront_bool_list = []

for value in X.waterfront:
    if value == 'YES':
        waterfront_bool_list.append(1)
    else:
        waterfront_bool_list.append(0)
        
X.waterfront = waterfront_bool_list

In [8]:
# convert view from string into categorical ordinal
view_rank_list = [] 
view_dict = {'NONE':0,'FAIR':1,'AVERAGE':2,'GOOD':3,'EXCELLENT':4}

for value in X.view:
    if value in list(view_dict.keys()):
        view_rank_list.append(view_dict[value])
    else:
        view_rank_list.append(0)
        
X.view = view_rank_list

In [9]:
# convert dates into ordinals 
X.date = pd.to_datetime(X['date'])
X.date = X['date'].map(dt.datetime.toordinal)

In [10]:
nulls = create_NaN_dictionary(X)

In [11]:
# funcitonize this ?? 

cor_df=X.corr().abs().stack().reset_index().sort_values(0, ascending=False)
cor_df['pairs'] = list(zip(cor_df.level_0, cor_df.level_1))
cor_df.set_index(['pairs'], inplace = True)
cor_df.drop(columns=['level_1', 'level_0'], inplace = True)
cor_df.columns = ['cc']
cor_df.drop_duplicates(inplace=True)

In [12]:
cor_df[(cor_df.cc>.60) & (cor_df.cc <1)]

,cc
pairs,
"(single_filing_bracket, joint_filing_bracket)",0.869690
"(sqft_living, sqft_above)",0.853341
"(sqft_living15, sqft_living)",0.736795
"(sqft_above, sqft_living15)",0.716207
"(sqft_living, bathrooms)",0.716197
"(sqft_lot, sqft_lot15)",0.707240
"(sqft_above, bathrooms)",0.638031
"(sqft_living, price)",0.619520


In [13]:
# droping joint_filing_bracket due to colinearity, will re-inspect the rest after one-hot encoding
X.drop('joint_filing_bracket',axis=1,inplace=True)

# renaming single_filing_bracket since it is now the only tax bracket feature and the name will be more idomatic
X.rename({'single_filing_bracket':'average_tax_bracket'},axis=1,inplace=True)

#### Engineering inferred features

##### ratios 
- bed bath ratio:           ratio of bedrooms to bathrooms
- level ratios:             ratio of square feet above 'grade' and below (ratio of everything else to the basement)
- live_lot_ratio:           ratio of living space to lot size

In [14]:
bbratios = []
lvl_ratios = []
live_lot_ratio = []

for index,row in X.iterrows(): #iterate through every record

    bbratio = row.bedrooms/row.bathrooms # calculate ratio of bedrooms to bathrooms
    bbratios.append(bbratio) # append ratio to the list

    LLratio = row.sqft_living/row.sqft_lot
    live_lot_ratio.append(LLratio)

    if row.sqft_basement == 0: # sqft_basement is zero if there is no basement
        lvl_ratios.append(0) # ratio should also be zero if there is no ratio
        
    else:
        lvl_ratio = row.sqft_above / row.sqft_basement # calculate ratio of space above grade vs below grade
        lvl_ratios.append(lvl_ratio) # append ratio to the list

X['bed_bath_ratio'] = bbratios # create new column and asign list as its values
X['level_ratio'] = lvl_ratios # create new column and asign list as its values
X['live_lot_ratio'] = live_lot_ratio # you get the idea . . . 

##### diferences 
- relative living space:    ratio of living space to the living space of the nearest 15 houses (sqft_living :: sqft_living15)
- relatve lot size:         same as living space but for lot size instead. 
- level difference:         difference in square footage of living space to basement space

In [15]:
rel_live_space = []
rel_lot_size = [] 
rel_difference = []

for index,row in X.iterrows(): # for every record 
    live_dif = row.sqft_living - row.sqft_living15 # calculate difference in sqft of the given house and the nearest 15 other houses
    rel_live_space.append(live_dif) # append it to the list

    lot_dif = row.sqft_lot - row.sqft_lot15 # calculate difference in sqft of the given lot and the nearest 15 other lots
    rel_lot_size.append(lot_dif) # append it to the list

    lvl_dif = row.sqft_above - row.sqft_basement # calculate difference between space above grade and below grade
    rel_difference.append(lvl_dif) # append it to the list

X['relative_living_space'] = rel_live_space # assign respective list to new column 
X['relative_lot_size'] = rel_lot_size
X['level_difference'] = rel_difference

#### Clean and one-hot encode categorical data

Using zipcodes to discover proximal waterfronts and one-hot encoding results

In [16]:
water_loc_dict = {'Duwamish':[98168],
'Elliott Bay':[98119,98104,98129,98132,98127,98125,98195,98101,98134,98170,98139,98131,98181], 
'Puget Sound':[98071,98083,98013,98070,98031,98131,98063,98195,98207,98190], 
'Lake Union':[98109], 
'Ship Canal':[00000], 
'Lake Washington':[98072,98077], 
'Lake Sammamish':[98074,98075,98029], 
'other lake':[00000], 
'river/slough waterfronts':[00000]}

# list to contain new column data
waterfront_list = []

# for loop to assign waterfront based on zipcode
for zipcode in X.zipcode:
    for k,v in water_loc_dict.items():
        if zipcode in v:
            waterfront_list.append(k)
            appended = True
            break
        else: 
            appended = False
    if not appended:
        waterfront_list.append('NONE')

# print(len(waterfront_list),set(waterfront_list))
X['waterfront_loc'] = waterfront_list

# create dummy variables
waterfront_dummies = pd.get_dummies(X.waterfront_loc).drop('NONE',axis=1).add_prefix('waterfront_')

# reasign X to concatenated dataframe, dropping features no longer needed
X = X.drop(['waterfront_loc','zipcode'],axis=1)
X = pd.concat([X,waterfront_dummies],axis=1)

Group Data by tax bracket and separate target variable

In [17]:
brackets = X.set_index(['average_tax_bracket']) 

In [18]:
y = brackets.price
y_log = np.log(y)

#### Inspect multicolinearity of features 

Based on variance inflation factors

In [19]:
good,bad = create_vif_list(brackets)
vif_dict = create_vif_dictionary(brackets)
vif_dict

{'const': 42738836.58948342,
 'date': 1.0080290934828011,
 'price': 2.2060728862151415,
 'bedrooms': 5.853952466616593,
 'bathrooms': 10.84789886361396,
 'sqft_living': inf,
 'sqft_lot': inf,
 'floors': 2.953092056279339,
 'waterfront': 1.1067408069132714,
 'view': 1.2103293132086186,
 'condition': 1.2414048300946874,
 'grade': 1.0981754823293823,
 'sqft_above': inf,
 'sqft_basement': inf,
 'yr_built': 2.3238414790353206,
 'yr_renovated': 1.1058092024836328,
 'sqft_living15': inf,
 'sqft_lot15': inf,
 'bed_bath_ratio': 7.343750843333311,
 'level_ratio': 1.1385391920633154,
 'live_lot_ratio': 2.308020253032007,
 'relative_living_space': inf,
 'relative_lot_size': inf,
 'level_difference': inf,
 'waterfront_Duwamish': 1.032434705624118,
 'waterfront_Elliott Bay': 1.0238292928846264,
 'waterfront_Lake Sammamish': 1.12357619890305,
 'waterfront_Lake Union': 1.0215981410019366,
 'waterfront_Lake Washington': 1.0509886825433166,
 'waterfront_Puget Sound': 1.0443308102758029}

In [20]:
X = brackets.drop(bad[1:],axis=1)

based on colinearity coefficient threshold of 60%

In [21]:
cor_df=X.corr().abs().stack().reset_index().sort_values(0, ascending=False)
cor_df['pairs'] = list(zip(cor_df.level_0, cor_df.level_1))
cor_df.set_index(['pairs'], inplace = True)
cor_df.drop(columns=['level_1', 'level_0'], inplace = True)
cor_df.columns = ['cc']
cor_df.drop_duplicates(inplace=True)
cor_df = cor_df.reset_index()

In [22]:
cor_df[(cor_df.cc>.60) & (cor_df.cc <1)]

,pairs,cc


##### Linear Transformation and Feature ranking 
using standardization (x - x_bar / sigma) to normalize units of measurement

investigate feature rankings using:
- stepwise method
- recursive elimination


In [ ]:
X_standardized = pd.DataFrame([]) # data standardized

for feature in X.columns:
    x_comp = (X[feature] - np.mean(X[feature])/np.std(X[feature])) # (x - x_bar)/sigma
    X_standardized[feature] = x_comp 

Create a baseline models for each tax bracket

In [40]:
base_model_dict = {
'two':run_OLS_model(X.loc[2,:],y_log[2]), 
'three':run_OLS_model(X.loc[3,:],y_log[3]), 
'four':run_OLS_model(X.loc[4,:],y_log[4]),
'five':run_OLS_model(X.loc[5,:],y_log[5]),
'six':run_OLS_model(X.loc[6,:],y_log[6]),
'seven':run_OLS_model(X.loc[7,:],y_log[7])
} 

bracket_call_dict = {
    'two':(X.loc[2,:],y_log[2]),
    'three':(X.loc[3,:],y_log[3]),
    'four':(X.loc[4,:],y_log[4]),
    'five':(X.loc[5,:],y_log[5]),
    'six':(X.loc[6,:],y_log[6]),
    'seven':(X.loc[7,:],y_log[7])
}


Stepwise selection method

In [50]:
stepped_model_selections_dict = {}

for k,v in bracket_call_dict.items():
    stepwisers = stepwise_selection(v[0],v[1],v[0].columns,threshold_in=0.01, threshold_out = 0.05, verbose=False)
    stepped_model_selections_dict[k] = stepwisers

stepped_model_dict = {}

for k,v in bracket_call_dict.items():
    selection = stepped_model_selections_dict[k]
    df = v[0][selection]
    stepped_model_dict[k] = run_OLS_model(df,v[1])


In [52]:
step_JB_dict = {}

for k,v in stepped_model_dict.items():
    resid = v.resid 
    name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
    test = sms.jarque_bera(resid)
    step_JB_dict[k] = list(zip(name, test))

In [55]:
for k in step_JB_dict:
    print(k,step_JB_dict[k])
    print('\n')

two [('Jarque-Bera', 15150.128436396091), ('Prob', 0.0), ('Skew', -3.1233978454614575), ('Kurtosis', 19.49288492514398)]


three [('Jarque-Bera', 32609.53321381114), ('Prob', 0.0), ('Skew', -2.116653974401343), ('Kurtosis', 9.867895781163663)]


four [('Jarque-Bera', 99259.17533762459), ('Prob', 0.0), ('Skew', -3.003494388487013), ('Kurtosis', 22.016193618606174)]


five [('Jarque-Bera', 142110.86060615702), ('Prob', 0.0), ('Skew', -6.467738389634934), ('Kurtosis', 86.20723545502742)]


six [('Jarque-Bera', 3032.2679458406933), ('Prob', 0.0), ('Skew', -2.7154215628084764), ('Kurtosis', 17.27435894862614)]


seven [('Jarque-Bera', 0.8525338585370891), ('Prob', 0.6529420295188015), ('Skew', 0.8516372406050681), ('Kurtosis', 2.286563614744315)]




In [49]:
RFECV_model_selections_dict = {}

for k,v in bracket_call_dict.items():
    recursives = run_RFECV(v[0],v[1],select_rank=True)
    RFECV_model_selections_dict[k] = recursives
    
RFECV_model_dict = {}

for k,v in bracket_call_dict.items():
    selection = RFECV_model_selections_dict[k]
    df = v[0][selection]
    RFECV_model_dict[k] = run_OLS_model(df,v[1])


In [56]:
RFECV_JB_dict = {}

for k,v in RFECV_model_dict.items():
    resid = v.resid 
    name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
    test = sms.jarque_bera(resid)
    RFECV_JB_dict[k] = list(zip(name, test))

In [57]:
for k in RFECV_JB_dict:
    print(k,RFECV_JB_dict[k])
    print('\n')

two [('Jarque-Bera', 14866.659475542403), ('Prob', 0.0), ('Skew', -3.1106307518964487), ('Kurtosis', 19.32525322386312)]


three [('Jarque-Bera', 32463.535282623045), ('Prob', 0.0), ('Skew', -2.113125172169822), ('Kurtosis', 9.851006031305701)]


four [('Jarque-Bera', 101179.42250548245), ('Prob', 0.0), ('Skew', -3.0216103769258655), ('Kurtosis', 22.206062285594673)]


five [('Jarque-Bera', 137242.0647769978), ('Prob', 0.0), ('Skew', -6.327335654930235), ('Kurtosis', 84.77833579860493)]


six [('Jarque-Bera', 3678.3014206837393), ('Prob', 0.0), ('Skew', -2.8047674715404782), ('Kurtosis', 18.858101426459353)]


seven [('Jarque-Bera', 0.26281178427745444), ('Prob', 0.8768617908428157), ('Skew', 0.19875876393105746), ('Kurtosis', 2.0548931530497674)]




In [ ]:
# r-squared, skew, kurt, Durbin-Watson, Jarque-Berra. 

In [ ]:
# plt.hist(stepped_model.resid)
# plt.show()

In [ ]:
# Q-Q plots 

for feature in X_standardized.columns:
    sm.qqplot(X_standardized[feature], line ='45')
    plt.title(feature)

plt.tight_layout()
plt.show()

In [ ]:
name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
test = sms.jarque_bera(resid)
list(zip(name, test))

Concat X_hot with predictor and export as CSV for use in diagnosis and validation notebook.

In [ ]:
export_df = pd.concat([y,X_hot],axis=1)

In [ ]:
# from pathlib import Path  
# filepath = Path('data/base_model.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# export_df.to_csv(filepath,index=False)